# Sentiment analysis of Twitter/X and Mastodon

In [21]:
%pip install tweepy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\jawad\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [36]:
import platform
import sys
from mastodon import Mastodon
import tweepy
from IPython.display import HTML
from datetime import datetime, timedelta, timezone
import pandas as pd
import re

### Mastadon Data Collection

In [23]:
Mastodon.create_app(
    'your_app_name',
    api_base_url = 'https://mastodon.social', #this is determined by your chosen server, if you picked the general server, don't change this line
    to_file = 'client_crediential_file_name.secret'
)

('gPWhhLaaUWJoIu2S9nCf9xbJLRcZAL4XhMidtDZp2CI',
 'HC8pUS3gTbJt47dpCkoDpzWfzVXFBsrFtxEZYKS1Wrg')

In [24]:
API = Mastodon(client_id = 'client_crediential_file_name.secret',)
url = API.auth_request_url(scopes=["read", "write", "follow", "push"])
print("Please visit this URL to authorize the app:", url)

Please visit this URL to authorize the app: https://mastodon.social/oauth/authorize?client_id=gPWhhLaaUWJoIu2S9nCf9xbJLRcZAL4XhMidtDZp2CI&response_type=code&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=read+write+follow+push&force_login=False&state=None&lang=None


In [25]:
code = input("Enter the code from the browser: ")

In [26]:
API.log_in(
    code=code,
    to_file = 'user_crediential_file_name.secret'
)
print("Successfully logged in. User access token saved in 'apple_user.secret'.")

Successfully logged in. User access token saved in 'apple_user.secret'.


In [31]:
q2 = "ChatGPT"
all_toots = []
off_set = 0

In [32]:
while True:
    # Fetch toots with the search API using offset
    search_results = API.search_v2(q=q2, result_type="statuses", offset=off_set)
    statuses = search_results["statuses"]

    if not statuses:
        break  # Exit loop if no more toots are returned

    # Add new toots to the collection
    all_toots.extend(statuses)

    earliest_toot = min(statuses, key=lambda x: x["created_at"])

    print(datetime.now(tz=timezone.utc) - earliest_toot["created_at"])
    
    if  timedelta(days=7) <= datetime.now(tz=timezone.utc) - earliest_toot["created_at"]:
        print("Reached 1 week")
        break

    # Update offset to fetch the next batch of results
    off_set += len(statuses)

    # Print progress
    print(f"Collected {len(statuses)} toots, total so far: {len(all_toots)}")

# Print results
print(f"Total toots collected: {len(all_toots)}")
if all_toots:
    print(f"Earliest toot: {all_toots[-1]['created_at']}")
    print(f"Latest toot: {all_toots[0]['created_at']}")

1:36:24.312582
Collected 15 toots, total so far: 15
2:47:20.574276
Collected 14 toots, total so far: 29
4:04:45.448861
Collected 17 toots, total so far: 46
9:40:35.145831
Collected 13 toots, total so far: 59
12:14:45.092846
Collected 14 toots, total so far: 73
12:14:45.752729
Collected 12 toots, total so far: 85
8:38:07.657702
Collected 12 toots, total so far: 97
12:09:55.429875
Collected 15 toots, total so far: 112
13:31:28.517630
Collected 18 toots, total so far: 130
15:23:02.468922
Collected 16 toots, total so far: 146
17:11:55.832130
Collected 16 toots, total so far: 162
1 day, 23:26:47.811333
Collected 16 toots, total so far: 178
1 day, 23:26:48.879081
Collected 14 toots, total so far: 192
21:21:12.944892
Collected 13 toots, total so far: 205
22:19:33.177247
Collected 19 toots, total so far: 224
1 day, 1:57:18.057193
Collected 15 toots, total so far: 239
1 day, 1:57:20.388993
Collected 18 toots, total so far: 257
1 day, 13:41:37.391820
Collected 19 toots, total so far: 276
1 day, 

In [37]:
needed_attributes = [
    'id', 'created_at', 'content', 'language', 'replies_count', 
    'reblogs_count', 'favourites_count', 'visibility', 'tags'
]

cleaned_toots = []
for toot in all_toots:
    cleaned_toot = {key: toot[key] for key in needed_attributes if key in toot}
    cleaned_toots.append(cleaned_toot)
    

In [38]:
def strip_html_tags(html_text):
    plain_text = re.sub(r'<.*?>', '', html_text)
    return plain_text

for toot in cleaned_toots:
    if 'content' in toot and toot['content']:  # Check if 'content' exists and is not None
        toot['content'] = strip_html_tags(toot['content'])

In [39]:
df = pd.DataFrame(cleaned_toots)
df

,id,created_at,content,language,replies_count,reblogs_count,favourites_count,visibility,tags
0,113600693934065544,2024-12-05 14:16:00+00:00,Você é um deles? ChatGPT chega 300 milhões de ...,pt,0,2,0,public,[]
1,113600685235169632,2024-12-05 14:28:11+00:00,"Meanwhile, in a tech leadership slack I'm in, ...",en,0,0,0,public,[]
2,113600649688291864,2024-12-05 14:19:08+00:00,For the lack of imagination of a better option...,en,1,0,0,public,[]
3,113600644820493090,2024-12-05 14:17:56+00:00,ChatGPT is a scab.,en,0,1,0,public,[]
4,113600644161311837,2024-12-05 14:17:46.229000+00:00,Um especialista em desinformação achou por bem...,None,0,0,0,public,[]
...,...,...,...,...,...,...,...,...,...
1732,113564457558167354,2024-11-29 04:55:02.879000+00:00,With chatgpt I made a silly calculator based o...,en,1,0,0,public,[]
1733,113564379239502973,2024-11-29 04:35:07.835000+00:00,Claude permet de choisir ou de créer un style ...,fr,0,0,1,public,"[{'name': 'ia', 'url': 'https://mastodon.socia..."
1734,113564333550579675,2024-11-28 14:15:15+00:00,5 Things You Must Not Share With AI Chatbotsht...,en,0,0,0,public,[]
1735,113564329141878768,2024-11-29 04:22:06+00:00,Anthropic’s Enhanced Writing Styles Anthro...,en,0,0,0,public,"[{'name': 'anthropic', 'url': 'https://mastodo..."


In [40]:
df.to_csv('mastodon_toots.csv', index=False)